# Walmart ,BestBuy GeoLocations-Analysis

In [1]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from census import Census
import gmaps
import warnings
warnings.filterwarnings('ignore')
# Google developer API key
from config_walmart import gkey
import folium
import time
# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:

url="http://api.walmartlabs.com/v1/stores?apiKey=rrvpb4x3az8mubaxka7uvhkj&city=California&format=json"
res=requests.get(url).json()

df=json_normalize(res)
df.head()

,city,coordinates,country,name,no,phoneNumber,stateProvCode,streetAddress,sundayOpen,timezone,zip
0,Fairfield,"[-122.0352432, 38.2790814]",US,Fairfield Supercenter,2048,707-428-4792,CA,2701 N Texas St,True,PST,94533
1,Ukiah,"[-123.196989, 39.133706]",US,Ukiah Store,2052,707-468-0258,CA,1155 Airport Park Blvd,True,PST,95482
2,Willows,"[-122.21772, 39.525898]",US,Willows Supercenter,2053,530-934-2054,CA,470 N Airport Rd,True,PST,95988
3,Long Beach,"[-118.147723, 33.86142]",US,Long Beach Supercenter,4101,424-296-6525,CA,3705 E South St,True,PST,90805
4,Jackson,"[-120.806192, 38.35793]",US,Jackson Store,2054,209-223-5384,CA,10355 Wicklow Way,True,PST,95642


In [3]:
l=df['coordinates']
z=[]
for i in l:
    for j in reversed(i):
        z.append(j)
N=2
latt=[]
lonn=[]
subList = [z[n:n+N] for n in range(0, len(z), N)]
for i in subList:
    latt.append(i[0])
    lonn.append(i[1])

location_df=pd.DataFrame({'lati':latt,
                         'longi':lonn})
location_df.head()

,lati,longi
0,38.279081,-122.035243
1,39.133706,-123.196989
2,39.525898,-122.217720
3,33.861420,-118.147723
4,38.357930,-120.806192


In [4]:
df_CA=df['name']
t=len(df_CA)
sc=0
s=0
nm=0
city='California'
for i in df_CA:
    if 'Supercenter' in i.split():
        sc=sc+1
    elif 'Store' in i.split():
        s=s+1
    else:
        nm=nm+1
print(f"Total number of Walmarts in {city}:{t}\nTotal number of Supercenters:{sc}\nTotal number of Stores:{s}\nTotal number of Neighborhood Markets:{nm}")

Total number of Walmarts in California:530
Total number of Supercenters:138
Total number of Stores:72
Total number of Neighborhood Markets:320


In [5]:

sc=[]
s=[]
nh=[]
for i in df_CA:
    if 'Supercenter' in i.split():
        sc.append(i)
    elif 'Store' in i.split():
        s.append(i)
    else:
        nh.append(i)
CAdf=df.set_index('name')
CAsc=CAdf.loc[sc]
CAs=CAdf.loc[s]
CAnh=CAdf.loc[nh]

sc_loc=CAsc['coordinates']
z=[]
for i in sc_loc:
    for j in reversed(i):
        z.append(j)
N=2
lat1=[]
lon1=[]
subList_sc = [z[n:n+N] for n in range(0, len(z), N)]
for i in subList_sc:
    lat1.append(i[0])
    lon1.append(i[1])

location_df_CA_sc=pd.DataFrame({'lati':lat1,
                         'longi':lon1})


s_loc=CAs['coordinates']
z=[]
for i in s_loc:
    for j in reversed(i):
        z.append(j)
N=2
lat2=[]
lon2=[]
subList_s = [z[n:n+N] for n in range(0, len(z), N)]
for i in subList_s:
    lat2.append(i[0])
    lon2.append(i[1])

location_df_CA_s=pd.DataFrame({'lati':lat2,
                         'longi':lon2})



nh_loc=CAnh['coordinates']
z=[]
for i in nh_loc:
    for j in reversed(i):
        z.append(j)
N=2
lat3=[]
lon3=[]
subList_nh = [z[n:n+N] for n in range(0, len(z), N)]
for i in subList_nh:
    lat3.append(i[0])
    lon3.append(i[1])

location_df_CA_nh=pd.DataFrame({'lati':lat3,
                         'longi':lon3})




CAsc.reset_index(inplace=True)
CAs.reset_index(inplace=True)
CAnh.reset_index(inplace=True)





In [6]:
stores =[]
for page in range(1,3):
    query ="https://api.bestbuy.com/v1/stores((region=CA))?apiKey=Jz4AJitfYbGIGGaOuTR09Lpl&show=city,country,lat,location,lng,name,longName,fullPostalCode,services,region,storeId,storeType,tradeIn&pageSize=100&page={}&format=json".format(page)
    st = requests.get(query).json()
    
    time.sleep(1)
    
    for store in st['stores']:
        stores.append(store)

In [7]:
storebestbuy_df = pd.DataFrame(stores)
storebestbuy_df=storebestbuy_df.fillna(0)
storebestbuy_df.head()

,city,country,fullPostalCode,lat,lng,longName,name,region,services,storeId,storeType,tradeIn
0,Hollywood,US,90028-6136,34.102020,-118.340431,BBM003HHCA,BBM003HHCA,CA,[],8900,Express Kiosk,0
1,West Hollywood,US,90038-2323,34.089130,-118.344470,West Hollywood,WEST HOLLYWOOD CA,CA,"[{'service': 'Amazon Alexa Experience'}, {'ser...",393,Big Box,0
2,Los Angeles,US,90039-1502,34.127770,-118.265300,Atwater Village,GLENDALE CA,CA,"[{'service': 'Apple Shop'}, {'service': 'Dyson...",183,Big Box,0
3,Los Angeles,US,90045-5803,33.945599,-118.392303,BBA009LAX4,BBA009LAX4,CA,[],8012,Express Kiosk,0
4,Los Angeles,US,90045-5803,33.945599,-118.392303,BBA010LAX7,BBA010LAX7,CA,[],8013,Express Kiosk,0


In [8]:
walmart_bestbuy_map = folium.Map(location=[37.229564,-120.047533], zoom_start=13) 

for i, j ,label in zip(location_df_CA_sc['lati'],location_df_CA_sc['longi'],CAsc['name']):
    folium.features.CircleMarker(
        [i,j],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(walmart_bestbuy_map)

for k,l,label in zip(location_df_CA_s['lati'],location_df_CA_s['longi'],CAs['name']):
    folium.features.CircleMarker(
        [k,l],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(walmart_bestbuy_map)



for m,n ,label in zip(storebestbuy_df['lat'],storebestbuy_df['lng'],storebestbuy_df['name']):
    folium.features.CircleMarker(
        [m,n],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(walmart_bestbuy_map)



# display map

walmart_bestbuy_map.save('storemap.html')
walmart_bestbuy_map

In [9]:
zip_code=[]
for index, row in storebestbuy_df.iterrows():

    zip = str.split(storebestbuy_df['fullPostalCode'][index], '-')
    zip_code.append(zip)

    
zip_again = pd.DataFrame(zip_code)

zip_again = zip_again.rename(columns={0: "zip"})
zip_again.head()

,zip,1
0,90028,6136
1,90038,2323
2,90039,1502
3,90045,5803
4,90045,5803


In [10]:
storebestbuy_df['zipcode']=zip_again['zip']
storebestbuy_df.head()

,city,country,fullPostalCode,lat,lng,longName,name,region,services,storeId,storeType,tradeIn,zipcode
0,Hollywood,US,90028-6136,34.102020,-118.340431,BBM003HHCA,BBM003HHCA,CA,[],8900,Express Kiosk,0,90028
1,West Hollywood,US,90038-2323,34.089130,-118.344470,West Hollywood,WEST HOLLYWOOD CA,CA,"[{'service': 'Amazon Alexa Experience'}, {'ser...",393,Big Box,0,90038
2,Los Angeles,US,90039-1502,34.127770,-118.265300,Atwater Village,GLENDALE CA,CA,"[{'service': 'Apple Shop'}, {'service': 'Dyson...",183,Big Box,0,90039
3,Los Angeles,US,90045-5803,33.945599,-118.392303,BBA009LAX4,BBA009LAX4,CA,[],8012,Express Kiosk,0,90045
4,Los Angeles,US,90045-5803,33.945599,-118.392303,BBA010LAX7,BBA010LAX7,CA,[],8013,Express Kiosk,0,90045


In [11]:
bestbuy_df=storebestbuy_df[['lat','lng','name','city']]
bestbuy_df.head()

,lat,lng,name,city
0,34.102020,-118.340431,BBM003HHCA,Hollywood
1,34.089130,-118.344470,WEST HOLLYWOOD CA,West Hollywood
2,34.127770,-118.265300,GLENDALE CA,Los Angeles
3,33.945599,-118.392303,BBA009LAX4,Los Angeles
4,33.945599,-118.392303,BBA010LAX7,Los Angeles


In [12]:
url1="http://api.walmartlabs.com/v1/stores?apiKey=rrvpb4x3az8mubaxka7uvhkj&city=SanJose&format=json"
res1=requests.get(url1).json()

df_SanJose=json_normalize(res1)
df_SanJose.head()

,city,coordinates,country,name,no,phoneNumber,stateProvCode,streetAddress,sundayOpen,timezone,zip
0,San Jose,"[-121.860387, 37.331034]",US,San Jose Supercenter,5435,408-885-1142,CA,777 Story Rd,True,PST,95122
1,San Jose,"[-121.871095, 37.31621]",US,San Jose Neighborhood Market,4174,408-454-4508,CA,1450 Monterey Rd,True,PST,95112
2,San Jose,"[-121.973182, 37.321668]",US,San Jose Neighborhood Market,2486,408-556-4505,CA,4080 Stevens Creek Blvd,True,PST,95129
3,San Jose,"[-121.880176, 37.255456]",US,San Jose Supercenter,5884,408-600-3072,CA,5095 Almaden Expy,True,PST,95118
4,San Jose,"[-121.773066, 37.313558]",US,San Jose Neighborhood Market,3023,408-826-0338,CA,4055 Evergreen Village Sq Ste 140,True,PST,95135


In [13]:
analyisi_sj_df=df_SanJose[['city','name','coordinates']]

In [14]:
location_df=analyisi_sj_df['coordinates']
cor=[]
for i in location_df:
    for j in reversed(i):
        cor.append(j)
N=2
lat_list=[]
lon_list=[]
subList_sj = [cor[n:n+N] for n in range(0, len(cor), N)]
for i in subList_sj:
    lat_list.append(i[0])
    lon_list.append(i[1])

#location_df_SanJose=pd.DataFrame({'lati':lat_list,
 #                        'longi':lon_list})
analyisi_sj_df['Latitude']=lat_list
analyisi_sj_df['Longitude']=lon_list

In [15]:
analyisi_sj_df

,city,name,coordinates,Latitude,Longitude
0,San Jose,San Jose Supercenter,"[-121.860387, 37.331034]",37.331034,-121.860387
1,San Jose,San Jose Neighborhood Market,"[-121.871095, 37.31621]",37.316210,-121.871095
2,San Jose,San Jose Neighborhood Market,"[-121.973182, 37.321668]",37.321668,-121.973182
3,San Jose,San Jose Supercenter,"[-121.880176, 37.255456]",37.255456,-121.880176
4,San Jose,San Jose Neighborhood Market,"[-121.773066, 37.313558]",37.313558,-121.773066
5,San Jose,San Jose Neighborhood Market,"[-121.9887, 37.291593]",37.291593,-121.988700
6,Santa Clara,Santa Clara Neighborhood Market,"[-121.9855022, 37.3900085]",37.390009,-121.985502
7,Milpitas,Milpitas Supercenter,"[-121.921182, 37.431416]",37.431416,-121.921182
8,Fremont,Fremont Store,"[-121.943195, 37.511302]",37.511302,-121.943195
9,Mountain View,Mountain View Store,"[-122.109748, 37.400846]",37.400846,-122.109748


In [16]:
CA_df=df[['city','name','coordinates']]
CA_df_sort=CA_df.groupby('city').count()
CA_df_sort=CA_df_sort.sort_values(by='name',ascending=False)
Top10=CA_df_sort.head(10)
Tail10=CA_df_sort.tail(10)

In [17]:
top_df=Top10.reset_index()
top_city=top_df['city']

In [18]:

population_df=pd.read_csv('population_data.csv')

In [19]:

population_df=population_df.rename(columns={'City':'city'})
population_df

In [20]:
Top10_city=Top10.reset_index()
Top10_city

,city,name,coordinates
0,Los Angeles,20,20
1,San Francisco,18,18
2,San Diego,18,18
3,Sacramento,14,14
4,San Jose,14,14
5,Bakersfield,7,7
6,Anaheim,7,7
7,Long Beach,6,6
8,Fresno,6,6
9,El Cajon,5,5


In [21]:
merg=pd.merge(Top10_city,population_df,on='city',how='inner')
merg

,city,name,coordinates,Population
0,Los Angeles,20,20,"3,928,864"
1,San Francisco,18,18,"852,469"
2,San Diego,18,18,"1,381,069"
3,Sacramento,14,14,"485,199"
4,San Jose,14,14,"1,015,785"
5,Bakersfield,7,7,"368,759"
6,Anaheim,7,7,"346,997"
7,Long Beach,6,6,"473,577"
8,Fresno,6,6,"515,986"
9,El Cajon,5,5,"103,091"


In [22]:
merg['median income']=['$62,289','$93,391','$74,569','$64,496','$92,379','$61,994','$63,281','$59,710','$45,554','$68,203']

In [23]:
merg['median income']=[i.replace('$','') for i in merg['median income']]
merg['median income']=[i.replace(',','') for i in merg['median income']]
merg['median income_downscale']=pd.to_numeric(merg['median income'])/1000
merg['median income_downscale']

0    62.289
1    93.391
2    74.569
3    64.496
4    92.379
5    61.994
6    63.281
7    59.710
8    45.554
9    68.203
Name: median income_downscale, dtype: float64

In [24]:

merg['Population']=[i.replace(',','') for i in merg['Population']]
merg['Population_downscale']=pd.to_numeric(merg['Population'])/100000
merg['Population']

0    3928864
1     852469
2    1381069
3     485199
4    1015785
5     368759
6     346997
7     473577
8     515986
9     103091
Name: Population, dtype: object

In [25]:
merg

,city,name,coordinates,Population,median income,median income_downscale,Population_downscale
0,Los Angeles,20,20,3928864,62289,62.289,39.28864
1,San Francisco,18,18,852469,93391,93.391,8.52469
2,San Diego,18,18,1381069,74569,74.569,13.81069
3,Sacramento,14,14,485199,64496,64.496,4.85199
4,San Jose,14,14,1015785,92379,92.379,10.15785
5,Bakersfield,7,7,368759,61994,61.994,3.68759
6,Anaheim,7,7,346997,63281,63.281,3.46997
7,Long Beach,6,6,473577,59710,59.710,4.73577
8,Fresno,6,6,515986,45554,45.554,5.15986
9,El Cajon,5,5,103091,68203,68.203,1.03091


In [49]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='dhanup369', api_key='mH49kPXAWBvyWrf1Xjyh')
trace1 = go.Bar(
        x=merg['city'], # assign x as the dataframe column 'x'
        y=merg['Population'],
       name='Population Count'
    )

trace2=go.Scatter(  x=merg['city'], # assign x as the dataframe column 'x'
        y=merg['Population'])
        


data=[trace1,trace2]

py.iplot(data, filename='bar-line')

url = py.plot(data, filename='City Vs Population bar-chart')

In [27]:
trace1 =go. Bar(
    x=merg['city'],
    y=merg['median income_downscale'],
    name='Income'
)
trace2 =go. Bar(
    x=merg['city'],
    y=merg['name'],
    name='Walmart Storecount'
)
trace3 =go. Scatter(
    x=merg['city'],
    y=merg['median income_downscale'],
    
)
trace4 =go. Scatter(
    x=merg['city'],
    y=merg['name'],
)




data1 = [trace1,trace2,trace3,trace4]
#layout = go.Layout(
#    barmode='group'
#)

#fig = go.Figure(data=data1, layout=layout)
py.iplot(data1, filename='bar-line')
#url = py.plot(data1, filename='City Vs Population bar-chart')

In [30]:
trace1 =go. Bar(
    x=merg['city'],
    y=merg['Population_downscale'],
    name='Population_downscale'
)
trace2 =go. Bar(
    x=merg['city'],
    y=merg['name'],
    name='Walmart Storecount'
)
trace3 =go.Scatter(
    x=merg['city'],
    y=merg['Population_downscale'],
    
)
trace4 =go.Scatter(
    x=merg['city'],
    y=merg['name'],
    
)


data1 = [trace1,trace2,trace3,trace4]
#layout = go.Layout(
 #   barmode='group'
#)

#fig = go.Figure(data=data1, layout=layout)
py.iplot(data1, filename='bar-line')
#url = py.plot(data1, filename='City Vs Population bar-chart')

In [31]:
storebestbuy_df.head()

,city,country,fullPostalCode,lat,lng,longName,name,region,services,storeId,storeType,tradeIn,zipcode
0,Hollywood,US,90028-6136,34.102020,-118.340431,BBM003HHCA,BBM003HHCA,CA,[],8900,Express Kiosk,0,90028
1,West Hollywood,US,90038-2323,34.089130,-118.344470,West Hollywood,WEST HOLLYWOOD CA,CA,"[{'service': 'Amazon Alexa Experience'}, {'ser...",393,Big Box,0,90038
2,Los Angeles,US,90039-1502,34.127770,-118.265300,Atwater Village,GLENDALE CA,CA,"[{'service': 'Apple Shop'}, {'service': 'Dyson...",183,Big Box,0,90039
3,Los Angeles,US,90045-5803,33.945599,-118.392303,BBA009LAX4,BBA009LAX4,CA,[],8012,Express Kiosk,0,90045
4,Los Angeles,US,90045-5803,33.945599,-118.392303,BBA010LAX7,BBA010LAX7,CA,[],8013,Express Kiosk,0,90045


In [32]:
best_buy=storebestbuy_df[['city','name','lat','lng']]

In [33]:
best_buy_sort=best_buy.groupby('city').count()
best_buy_sort=best_buy_sort.sort_values(by='name',ascending=False)
#Top10bb=best_buy_sort.head(10)
#Tail10bb=best_buy_sort.tail(10)
Top10bb=best_buy_sort.head(10)
Top10bb

,name,lat,lng
city,,,
Los Angeles,10,10,10
San Diego,9,9,9
San Francisco,6,6,6
San Jose,4,4,4
Oakland,3,3,3
Sacramento,3,3,3
San Rafael,2,2,2
Irvine,2,2,2
South San Francisco,2,2,2


In [34]:
top_dfbb=Top10bb.reset_index()
top_citybb=top_dfbb['city']

In [35]:
Top10_citybb=Top10bb.reset_index()
Top10_citybb

,city,name,lat,lng
0,Los Angeles,10,10,10
1,San Diego,9,9,9
2,San Francisco,6,6,6
3,San Jose,4,4,4
4,Oakland,3,3,3
5,Sacramento,3,3,3
6,San Rafael,2,2,2
7,Irvine,2,2,2
8,South San Francisco,2,2,2
9,Santa Rosa,2,2,2


In [36]:
mergbb=pd.merge(Top10_citybb,population_df,on='city',how='inner')
mergbb

,city,name,lat,lng,Population
0,Los Angeles,10,10,10,"3,928,864"
1,San Diego,9,9,9,"1,381,069"
2,San Francisco,6,6,6,"852,469"
3,San Jose,4,4,4,"1,015,785"
4,Oakland,3,3,3,"413,775"
5,Sacramento,3,3,3,"485,199"
6,San Rafael,2,2,2,"59,237"
7,Irvine,2,2,2,"248,531"
8,South San Francisco,2,2,2,"67,009"
9,Santa Rosa,2,2,2,"174,170"


In [37]:
mergbb['median income']=['$62,289','$74,569','$93,391','$92,379','$60,472','$57,200','$96,660','$109,956','$89,186','$69,921']

In [38]:
mergbb

,city,name,lat,lng,Population,median income
0,Los Angeles,10,10,10,"3,928,864","$62,289"
1,San Diego,9,9,9,"1,381,069","$74,569"
2,San Francisco,6,6,6,"852,469","$93,391"
3,San Jose,4,4,4,"1,015,785","$92,379"
4,Oakland,3,3,3,"413,775","$60,472"
5,Sacramento,3,3,3,"485,199","$57,200"
6,San Rafael,2,2,2,"59,237","$96,660"
7,Irvine,2,2,2,"248,531","$109,956"
8,South San Francisco,2,2,2,"67,009","$89,186"
9,Santa Rosa,2,2,2,"174,170","$69,921"


In [39]:
mergbb['median income']=[i.replace('$','') for i in mergbb['median income']]
mergbb['median income']=[i.replace(',','') for i in mergbb['median income']]
mergbb['median income_downscale']=pd.to_numeric(mergbb['median income'])/1000
mergbb['median income_downscale']

0     62.289
1     74.569
2     93.391
3     92.379
4     60.472
5     57.200
6     96.660
7    109.956
8     89.186
9     69.921
Name: median income_downscale, dtype: float64

In [40]:
mergbb['Population']=[i.replace(',','') for i in mergbb['Population']]
mergbb['Population_downscale']=pd.to_numeric(mergbb['Population'])/100000
mergbb['Population_downscale']

0    39.28864
1    13.81069
2     8.52469
3    10.15785
4     4.13775
5     4.85199
6     0.59237
7     2.48531
8     0.67009
9     1.74170
Name: Population_downscale, dtype: float64

In [41]:
trace1 = go.Bar(
        x=mergbb['city'], # assign x as the dataframe column 'x'
        y=mergbb['Population'],
       name='Population'
       
    )

trace2=go.Scatter(  x=mergbb['city'], # assign x as the dataframe column 'x'
        y=mergbb['Population'])
        


data=[trace1,trace2]

py.iplot(data, filename='bar-line')



In [42]:
trace1 =go. Bar(
    x=mergbb['city'],
    y=mergbb['median income_downscale'],
    name='Income'
)
trace2 =go. Bar(
    x=mergbb['city'],
    y=mergbb['name'],
    name='BestBuy Storecount'
)
trace3 =go. Scatter(
    x=mergbb['city'],
    y=mergbb['median income_downscale'],
    
)
trace4 =go. Scatter(
    x=mergbb['city'],
    y=mergbb['name'],
    
)



data1 = [trace1,trace2,trace3,trace4]
#layout = go.Layout(
#    barmode='group'
#)
py.iplot(data1,filename='bar_line')
#fig = go.Figure(data=data1, layout=layout)
#py.iplot(fig, filename='grouped-bar')
#url = py.plot(data1, filename='City Vs Population bar-chart')

In [43]:
trace1 =go. Bar(
    x=mergbb['city'],
    y=mergbb['Population_downscale'],
    name='Population_downscale'
)
trace2 =go. Bar(
    x=mergbb['city'],
    y=mergbb['name'],
    name='BestBuy Storecount'
)


trace3 =go. Scatter(
    x=mergbb['city'],
    y=mergbb['Population_downscale'],
    
)
trace4 =go.Scatter(
    x=mergbb['city'],
    y=mergbb['name'],
    
)


data1 = [trace1,trace2,trace3,trace4]
py.iplot(data1,filename='bar_line')

#fig = go.Figure(data=data1, layout=layout)
#py.iplot(fig, filename='grouped-bar')
#url = py.plot(data1, filename='City Vs Population bar-chart')

In [44]:
merge_walmart_bestbuy=pd.merge(merg,mergbb,on='city',how='inner')

In [45]:
merge_walmart_bestbuy=merge_walmart_bestbuy[['city','name_x','Population_downscale_x','median income_downscale_x','name_y']]

In [46]:
merge_walmart_bestbuy=merge_walmart_bestbuy.rename(columns={'name_x':'Walmart',
                                     'name_y':'BestBuy',
                                     'Population_downscale_x':'Population',
                                     'median income_downscale_x':'Income'
                                      })

merge_walmart_bestbuy=merge_walmart_bestbuy[['city','Walmart','BestBuy','Population','Income']]

In [47]:
merge_walmart_bestbuy

,city,Walmart,BestBuy,Population,Income
0,Los Angeles,20,10,39.28864,62.289
1,San Francisco,18,6,8.52469,93.391
2,San Diego,18,9,13.81069,74.569
3,Sacramento,14,3,4.85199,64.496
4,San Jose,14,4,10.15785,92.379


In [48]:
trace1 =go. Bar(
    x=merge_walmart_bestbuy['city'],
    y=merge_walmart_bestbuy['Walmart'],
    name='WalmartCount'
)

trace2 =go. Bar(
    x=merge_walmart_bestbuy['city'],
    y=merge_walmart_bestbuy['BestBuy'],
    name='BestBuyCount'
)
trace3 =go.Scatter(
    x=merge_walmart_bestbuy['city'],
    y=merge_walmart_bestbuy['Walmart'],
    
)

trace4 =go. Scatter(
    x=merge_walmart_bestbuy['city'],
    y=merge_walmart_bestbuy['BestBuy'],
    
)



data1 = [trace1,trace2,trace3,trace4]



#fig = go.Figure(data=data1, layout=layout)
py.iplot(data1, filename='bar_line')
#url = py.plot(data1, filename='Walmart Vs BestBuy')